In [ ]:
%load_ext autoreload
%autoreload 2

# Auto Correct with Minimum Edit Distance

In this notebook, we will implement a simple auto-correct system. We will use the Minimum Edit Distance algorithm to measure the similarity between a word and a target word. Then, we will use this to find the word in our vocabulary that is most similar to the input.

We will implement models that correct words that are 1 and 2 edit distances away. 

An edit could consist of one of the following options: 

- Delete (remove a letter): `hat` => `at, ha, ht`
- Switch (swap 2 adjacent letters): `eta` => `eat, tea,...`
- Replace (change 1 letter to another): `jat` => `hat, rat, cat, mat, ...`
- Insert (add a letter): `te` => `the, ten, ate, ...`

You will be using the four methods above to implement an Auto-correct. 
- To do so, you will need to compute probabilities that a certain word is correct given an input. 

> This auto-correct you are about to implement was first created by [Peter Norvig](https://en.wikipedia.org/wiki/Peter_Norvig) in 2007. His [original article](https://norvig.com/spell-correct.html) may be a useful reference for this notebook.

## Preprocessing 

In [ ]:
import re
from collections import Counter
import numpy as np
import pandas as pd

As in any other machine learning task, the first thing you have to do is process your data set.

Your first task is to read in a file called **`shakespeare.txt`** which is found in `notebooks/data` directory.

> Hint: use `re.findall()` to extract all the words from a string.

In [ ]:
def process_data(file_name: str) -> list[str]:
    """
    Process the data from a given file and return a list of all words in the corpus.

    The words should be lowercased and should not contain any punctuation.

    Args:
        file_name (str): the file name of the data

    Returns:
        list[str]: a list of words
    """

    # TODO implement this function
    raise NotImplementedError("This function needs to be implemented.")

In [ ]:
word_list = process_data("data/shakespeare.txt")
VOCABULARY = set(word_list)

print(word_list[0:10])
print(len(VOCABULARY))

**Expected Output**

```txt
['o', 'for', 'a', 'muse', 'of', 'fire', 'that', 'would', 'ascend', 'the']
6116
```

Implement a `get_word_frequencies()` which returns a dictionary where the key is a word and the value is the number of times the word appears in the list.  


In [ ]:
def get_word_frequencies(words: list[str]) -> dict[str, int]:
    """
    Count the number of occurrences of each word in a given list of words.

    Args:
        words (list[str]): a list of words

    Returns:
        dict[str, int]: a dictionary that maps from words to counts
    """

    # TODO implement this function
    raise NotImplementedError("This function needs to be implemented.")

In [ ]:
word_count_dict = get_word_frequencies(word_list)
print(len(word_count_dict))
print(word_count_dict.get("thee", 0))

Expected Output

```txt
6116
240
```

Given the dictionary of word counts, compute the probability that each word will appear if randomly selected from the corpus of words.

$$P(w_i) = \frac{C(w_i)}{m} \tag{Eqn-2}$$

Implement `get_word_probabilities()` function which gives you the probability that a word occurs in a sample. This returns a dictionary where the keys are words, and the value for each word is its probability in the corpus of words.

In [ ]:
def get_word_probabilities(freqs: dict[str, int]) -> dict[str, float]:
    """
    Compute the probability of each word in a given dictionary of word frequencies.

    Args:
        freqs (dict[str, int]): a dictionary that maps from words to counts

    Returns:
        dict[str, float]: a dictionary that maps from words to probabilities
    """

    # TODO implement this function
    raise NotImplementedError("This function needs to be implemented.")

In [ ]:
WORD_PROBABILITIES = get_word_probabilities(word_count_dict)
print(len(WORD_PROBABILITIES))
print(f"{WORD_PROBABILITIES['thee']:.4f}")

Expected Output

```txt
6116
0.0045
```

## Edit Operations

Now, that you have computed $P(w_i)$ for all the words in the corpus, you will write a few functions to manipulate strings so that you can edit the erroneous strings and return the right spellings of the words. In this section, you will implement four functions: 

* `delete_letter()`: given a word, it returns all the possible strings that have **one character removed**. 
* `replace_letter()`: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* `insert_letter()`: given a word, it returns all the possible strings that have an **additional character inserted**.

Implement a `delete_letter()` function that, given a word, returns a list of strings with one character deleted. 

For example, given the word `nice`, it would return the set: `['ice', 'nce', 'nic', 'nie']`.


In [ ]:
def delete_letter(word: str) -> list[str]:
    """
    Performs a delete operation on the given word and returns all possible strings that can occur after the delete operation.

    Args:
        word (str): a word on which to perform the deletion operation

    Returns:
        list[str]: a list of string with one character deleted
    """

    # TODO implement this function
    raise NotImplementedError("This function needs to be implemented.")

In [ ]:
print(delete_letter("cans"))
print(delete_letter("at"))

Expected Output

```txt
['ans', 'cns', 'cas', 'can']
['t', 'a']
```

Now implement the `replace_letter()` function.

The function replaces one letter from the original word with another letter from the latin alphabet. It should return a list of all possible words obtained by replacing one letter from the original word with another letter from the alphabet. Note that the original word should **not** be in the returned set.

In [ ]:
def replace_letter(word: str) -> list[str]:
    """
    Performs a replace operation on the given word and returns all possible strings that can occur after the replace operation.

    As replacement characters, you should use the characters from the standard English alphabet of 26 characters.

    Args:
        word (str): a word on which to perform the replacement operation

    Returns:
        list[str]: a list of strings with each character replaced
    """

    # TODO implement this function
    raise NotImplementedError("This function needs to be implemented.")

In [ ]:
replace_letter_l = replace_letter("can")
print(len(replace_letter_l))
print("can" in replace_letter_l)

Expected Output: 

```txt
75
False
```

> Note how the input word 'can' should not be one of the output words.

Now implement a function that takes in a word and returns a list with a letter inserted at every offset.

Note that the original word should **not** be in the returned set.

In [ ]:
def insert_letter(word: str) -> list[str]:
    """
    Performs an insert operation on the given word and returns all possible strings that can occur after the insert operation.

    As insertion characters, you should use the characters from the standard English alphabet of 26 characters.

    Args:
        word (str): a word on which to perform the insertion operation

    Returns:
        list[str]: a list of strings with a new character inserted
    """

    # TODO implement this function
    raise NotImplementedError("This function needs to be implemented.")

In [ ]:
insert_l = insert_letter("at")
print(len(insert_l))

Expected output

```txt
78
```

## Combining the edits

Now that you have implemented the string manipulations, you will create two functions that, given a string, will return all the possible single and double edits on that string. These will be `edit_one_letter()` and `edit_two_letters()`.

Implement the `edit_one_letter()` function to get all the possible edits that are one edit away from a word. The edits  consist of the replace, insert, delete, and optionally the switch operation. You should use the previous functions you have already implemented to complete this function. 

Note that those functions return *lists* while this function should return a *python set*. Utilizing a set eliminates any duplicate entries.

In [ ]:
def edit_one_letter(word: str) -> set[str]:
    """
    Given a word, return a set of strings with one possible edit on the given word.

    Args:
        word (str): a word on which to perform the edit operation

    Returns:
        set[str]: a set of strings with one possible edit on the given word
    """

    # TODO implement this function
    raise NotImplementedError("This function needs to be implemented.")

In [ ]:
print(len(edit_one_letter("at")))

Expected Output

```txt
128
```

Now you can generalize this to implement to get two edits on a word. To do so, you would have to get all the possible edits on a single word and then for each modified word, you would have to modify it again. 

Implement the `edit_two_letters()` function that returns a set of words that are two edits away. Note that creating additional edits based on the `edit_one_letter` function may 'restore' some one_edits to zero or one edits. That is allowed here. This accounted for in get_corrections.

In [ ]:
def edit_two_letters(word: str) -> set[str]:
    """
    Given a word, return a set of strings with two possible edits on the given word.

    Args:
        word (str): a word on which to perform the edit operation

    Returns:
        set[str]: a set of strings with two possible edits on the given word
    """

    # TODO implement this function
    raise NotImplementedError("This function needs to be implemented.")

In [ ]:
print(len(edit_two_letters("a")))
print(len(edit_two_letters("at")))

Expected Output

```txt
2654
7130
```

## Suggest Possible Corrections

Now you will use your `edit_two_letters()` function to get a set of all the possible 2 edits on your word. You will then use those strings to get the most probable word you meant to type aka your typing suggestion.

Implement `get_corrections()`, which returns a list of zero to n possible suggestion tuples of the form (word, probability_of_word). 

Generate suggestions for a supplied word: You'll use the edit functions you have developed. The suggestion algorithm should work as follows:

- If the word is in the vocabulary, suggest the word. 
- Otherwise, if there are suggestions from `edit_one_letter()` that are in the vocabulary, use those. 
- Otherwise, if there are suggestions from `edit_two_letters()` that are in the vocabulary, use those. 
- Otherwise, suggest the input word.*  
- The idea is that words generated from fewer edits are more likely than words with more edits.

> Edits of one or two letters may 'restore' strings to either zero or one edit. This algorithm accounts for this by preferentially selecting lower distance edits first.

In [ ]:
def get_corrections(
    word: str,
    probs: dict[str, float] = WORD_PROBABILITIES,
    vocab: set[str] = VOCABULARY,
    n: int = 2,
) -> list[tuple[str, float]]:
    """
    Given a word, return a list of n possible spelling corrections ranked by probability.

    Args:
        word (str): the word to be corrected
        probs (dict[str, float]): a dictionary that maps from words to probabilities
        vocab (set[str]): a set of all words in the vocabulary
        n (int): the number of possible spelling corrections to return

    Returns:
        list[tuple[str, float]]: a list of tuples with the n possible spelling corrections and their probabilities, sorted in descending order of probabilities
    """

    # TODO implement this function
    raise NotImplementedError("This function needs to be implemented.")

In [ ]:
# Test your implementation - feel free to try other words in my word
tmp_corrections = get_corrections("dys")
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

Expected Output

```txt
word 0: days, probability 0.000410
word 1: dye, probability 0.000019
```

In [ ]:
get_corrections("imagunary")

## Minimum Edit distance

Now that you have implemented your auto-correct, how do you evaluate the similarity between two strings? For example: `waht` and `what`

Also how do you efficiently find the shortest path to go from the word, `waht` to the word `what`?

You will implement a **dynamic programming** algorithm that will tell you the minimum number of edits required to convert a string into another string.

Dynamic Programming breaks a problem down into subproblems which can be combined to form the final solution. 

Here, given two strings $source$ and $target$, we will compute all the combinations of substrings, and calculate their edit distance.

To do this efficiently, we will use a table to maintain the previously computed substrings and use those to calculate larger substrings.

You have to create a matrix and update each element in the matrix as follows:

Initialization:

$$
\begin{align}
D[0,0] &= 0 \\
D[i,0] &= D[i-1,0] + del\_cost(source[i]) \tag{4}\\
D[0,j] &= D[0,j-1] + ins\_cost(target[j]) \\
\end{align}
$$

Per Cell Operations

$$
\begin{align}
 \\
D[i,j] =min
\begin{cases}
D[i-1,j] + del\_cost\\
D[i,j-1] + ins\_cost\\
D[i-1,j-1] + \left\{\begin{matrix}
rep\_cost; & if src[i]\neq tar[j]\\
0 ; & if src[i]=tar[j]
\end{matrix}\right.
\end{cases}
\tag{5}
\end{align}
$$

So converting the source word `play` to the target word `stay`, using an input cost of one, a delete cost of `1`, and replace cost of `2` would give you the following table:

|  | # | s | t | a | y |
|---|---|---|---|---|---|
| **#** | 0 | 1 | 2 | 3 | 4 |
| **p** | 1 | 2 | 3 | 4 | 5 |
| **l** | 2 | 3 | 4 | 5 | 6 |
| **a** | 3 | 4 | 5 | 4 | 5 |
| **y** | 4 | 5 | 6 | 5 | 4 |

Implement the `min_edit_distance()` function below to get the minimum amount of edits required given a source string and a target string. 

In [ ]:
def min_edit_distance(
    source: str, target: str, ins_cost: int = 1, del_cost: int = 1, rep_cost: int = 2
) -> tuple[np.ndarray, int]:
    """
    Compute the minimum edit distance between two strings.

    Args:
        source (str): the source string
        target (str): the target string
        ins_cost (int): the cost of an insertion
        del_cost (int): the cost of a deletion
        rep_cost (int): the cost of a replacement

    Returns:
        tuple[np.ndarray, int]: a tuple containing the cost matrix and the minimum edit distance
    """

    # TODO implement this function
    raise NotImplementedError("This function needs to be implemented.")

Test your implementation:

In [ ]:
# get the edit distance
source = "play"
target = "stay"
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ", min_edits, "\n")

# visualize the matrix
idx = list("#" + source)
cols = list("#" + target)
df = pd.DataFrame(matrix, index=idx, columns=cols)
print(df)

**Expected Results:**

minimum edits:  4

```txt
   #  s  t  a  y
#  0  1  2  3  4
p  1  2  3  4  5
l  2  3  4  5  6
a  3  4  5  4  5
y  4  5  6  5  4
```

In [ ]:
# get the edit distance
source = "eer"
target = "near"
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ", min_edits, "\n")

# visualize the matrix
idx = list("#" + source)
cols = list("#" + target)
df = pd.DataFrame(matrix, index=idx, columns=cols)
print(df)

**Expected Results**

```txt
minimum edits:  3 

   #  n  e  a  r
#  0  1  2  3  4
e  1  2  1  2  3
e  2  3  2  3  4
r  3  4  3  4  3
```